In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
image_size = (256, 256)
batch_size = 128
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data//training_set',
    validation_split=0.2,
    color_mode='rgb',
    label_mode='binary',
    subset='training',
    seed=13372,
    image_size=image_size,
    batch_size=batch_size)

train_size = int(0.8 * dataset.cardinality().numpy())
valid_size = dataset.cardinality().numpy() - train_size
train_ds = dataset.take(train_size)
valid_ds = dataset.skip(train_size)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [1]:
def cnn_model3(input_shape, num_classes):
  model = tf.keras.Sequential()
  # Add input layer with input_shape and float32 dtype
  model.add(tf.keras.layers.InputLayer(input_shape=input_shape, dtype='float32'))
  # Add 5 convolutional layers with 32, 64, 128, 256, and 512 filters
  model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
  model.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu'))
  
  # Add max pooling layer after each convolutional layer
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  # Flatten the output of the convolutional layers
  model.add(tf.keras.layers.Flatten())
  
  # Add 2 fully connected layers with 512 and 128 units, respectively
  model.add(tf.keras.layers.Dense(512, activation='sigmoid'))
  model.add(tf.keras.layers.Dense(128, activation='sigmoid'))
  
  if num_classes == 2:
    activation = "sigmoid"
    units = 1
  else:
    activation = "softmax"
    units = num_classes
    
  model.add(layers.Dropout(0.5))
  # Add output layer with num_classes units and softmax activation
  model.add(tf.keras.layers.Dense(units, activation=activation))
  
  return model

In [ ]:
input_shape = (256, 256, 3)
num_classes = 2
model = cnn_model2(input_shape, num_classes)

In [ ]:
model.summary()

In [ ]:
epochs = 5

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds
)

In [ ]:
img = keras.preprocessing.image.load_img(
    "data//training_set//cats//cat.4185.jpg", target_size=input_shape
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(predictions[0])
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")